Adapted from `rlcard/examples/leduc_holdem_dqn_pytorch.py`

In [1]:
import os
import sys
sys.path.insert(0, os.path.abspath('./rlcard'))

In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
import torch
import rlcard
from DQNAgent import DQNAgent
from rlcard.agents import RandomAgent, CFRAgent
from rlcard.utils import set_global_seed, tournament
from rlcard.utils import Logger

In [4]:
# Make environment
env = rlcard.make('kuhn-poker', config={'seed': 0})
eval_env = rlcard.make('kuhn-poker', config={'seed': 0})

In [5]:
# Set the iterations numbers and how frequently we evaluate the performance
evaluate_every = 1_000
evaluate_num = 10_000
episode_num = 100_000

In [6]:
# The paths for saving the logs and learning curves
log_dir = './experiments/kuhn_poker_dqn_result/'

In [7]:
# Set a global seed
set_global_seed(0)

In [8]:
agent = DQNAgent(
    env.action_num,
    env.state_shape[0],
    memory=2_000_000,
    lr=0.1,
    batch_size=128,
    update_every=64,
    eps_start=0.12,
    target_update=300,
    hidden_neurons=[64]
)

cfr_agent = CFRAgent(eval_env, model_path='./models/kuhn_poker_cfr')
cfr_agent.load()

random_agent = RandomAgent(action_num=eval_env.action_num)

env.set_agents([agent, agent])
eval_env.set_agents([agent, cfr_agent])

In [9]:
# Init a Logger to plot the learning curve
logger = Logger(log_dir)

In [10]:
for episode in range(episode_num):
    # Generate data from the environment
    trajectories, _ = env.run(is_training=True)

    # Feed transitions into agent memory, and train the agent
    for ts in trajectories[0]:
        agent.train(ts)

    # Evaluate the performance. Play with CFR agents.
    if episode % evaluate_every == 0:
        print(episode / evaluate_every)
        logger.log_performance(agent.weight_updates,
                               tournament(eval_env, evaluate_num)[0])

0.0

----------------------------------------
  timestep     |  0
  reward       |  -0.252
----------------------------------------
1.0

----------------------------------------
  timestep     |  14
  reward       |  -0.1784
----------------------------------------
2.0

----------------------------------------
  timestep     |  33
  reward       |  -0.0284
----------------------------------------
3.0

----------------------------------------
  timestep     |  51
  reward       |  0.0045
----------------------------------------
4.0

----------------------------------------
  timestep     |  69
  reward       |  0.0168
----------------------------------------
5.0

----------------------------------------
  timestep     |  88
  reward       |  0.0125
----------------------------------------
6.0

----------------------------------------
  timestep     |  105
  reward       |  -0.0492
----------------------------------------
7.0

----------------------------------------
  timestep     |  12

61.0

----------------------------------------
  timestep     |  986
  reward       |  0.0042
----------------------------------------
62.0

----------------------------------------
  timestep     |  1002
  reward       |  -0.0599
----------------------------------------
63.0

----------------------------------------
  timestep     |  1018
  reward       |  -0.0148
----------------------------------------
64.0

----------------------------------------
  timestep     |  1033
  reward       |  -0.0789
----------------------------------------
65.0

----------------------------------------
  timestep     |  1048
  reward       |  -0.07
----------------------------------------
66.0

----------------------------------------
  timestep     |  1064
  reward       |  -0.0489
----------------------------------------
67.0

----------------------------------------
  timestep     |  1079
  reward       |  0.008
----------------------------------------
68.0

----------------------------------------


In [11]:
# Close files in the logger
logger.close_files()

In [12]:
# Plot the learning curve
logger.plot('DQN')

./experiments/kuhn_poker_dqn_result/performance.csv


In [13]:
# Save model
save_dir = 'models/kuhn_poker_dqn'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

state_dict = agent.get_state_dict()

torch.save(state_dict, os.path.join(save_dir, 'model.pth'))

In [14]:
eval_env_dqn_dqn = rlcard.make('kuhn-poker', config={'seed': 1})
eval_env_dqn_dqn.set_agents([agent, agent])
payoffs = tournament(eval_env_dqn_dqn, 10000)
print(payoffs)

[-0.0012, 0.0012]


In [15]:
eval_env_dqn_dqn = rlcard.make('kuhn-poker', config={'seed': 1})
eval_env_dqn_dqn.set_agents([agent, random_agent])
payoffs = tournament(eval_env_dqn_dqn, 10000)
print(payoffs)

[0.2857, -0.2857]


In [16]:
eval_env_dqn_dqn = rlcard.make('kuhn-poker', config={'seed': 1})
eval_env_dqn_dqn.set_agents([agent, cfr_agent])
payoffs = tournament(eval_env_dqn_dqn, 100000)
print(payoffs)

[0.00159, -0.00159]
